In [2]:
import familyanalyzer as fa
XML = '/cs/research/external/home/naglover/HOGscore/HOG0152989.orthoxml+'

In [49]:
#Parse OrthoXML file, extract taxonomy, augment taxonomy
op = fa.OrthoXMLParser(XML)  
tax = fa.TaxonomyFactory.newTaxonomy(op)
op.augmentTaxonomyInfo(tax)



Adding missing taxonomy annotation: 100% || Elapsed Time: 0:00:00 Time: 0:00:00


In [52]:
tax.get_histories(op) # Generate FamHistory objects to store gene lists
tax.get_comparisons(op)
gtt = fa.GeneTreeTracer(op, tax)
tax_node = tax[tax.root]
for hog in hoglist:
    gene_tree = gtt.trace_gene_family(tax_node, hog, None, False)
newick_long = gene_tree.write()

Getting histories: 100% |#################| Elapsed Time: 0:00:00 Time: 0:00:00
Comparing: 100% |#########################| Elapsed Time: 0:00:00 Time: 0:00:00


In [65]:
print(tax.newick())

(("Echinops telfairi"[&&NHX:Genes=1:Identical=1:Duplicated=0:Lost=0:Novel=0:Singleton=0], "Loxodonta africana"[&&NHX:Genes=1:Identical=1:Duplicated=0:Lost=0:Novel=0:Singleton=0])Afrotheria[&&NHX:Genes=1:Identical=1:Duplicated=0:Lost=0:Novel=0:Singleton=0], (((((("Macaca mulatta"[&&NHX:Genes=1:Identical=1:Duplicated=0:Lost=0:Novel=0:Singleton=0], ("Pongo abelii"[&&NHX:Genes=1:Identical=1:Duplicated=0:Lost=0:Novel=0:Singleton=0], ("Homo sapiens"[&&NHX:Genes=3:Identical=0:Duplicated=3:Lost=0:Novel=0:Singleton=0], "Pan troglodytes"[&&NHX:Genes=1:Identical=1:Duplicated=0:Lost=0:Novel=0:Singleton=0])Homininae[&&NHX:Genes=1:Identical=1:Duplicated=0:Lost=0:Novel=0:Singleton=0])Hominidae[&&NHX:Genes=1:Identical=1:Duplicated=0:Lost=0:Novel=0:Singleton=0])Catarrhini[&&NHX:Genes=1:Identical=1:Duplicated=0:Lost=0:Novel=0:Singleton=0], "Callithrix jacchus"[&&NHX:Genes=1:Identical=1:Duplicated=0:Lost=0:Novel=0:Singleton=0])Simiiformes[&&NHX:Genes=1:Identical=1:Duplicated=0:Lost=0:Novel=0:Singleton=0]

In [53]:
newick_short = re.sub(r'\[.*?\]', '',newick_long)
print(newick_short)
with open("/cs/research/external/home/naglover/HOGscore/testHOG.hog.tree", "w") as f:
    f.write(newick_short)

((ECHTE05596, LOXAF05609)Afrotheria, ((((((MACMU20172, (PONAB04505, ((HUMAN00001, HUMAN09883, HUMAN13752)DUP_1, PANTR18725)Homininae)Hominidae)Catarrhini, CALJA02046)Simiiformes, TARSY05187)Haplorrhini, OTOGA00136)Primates, ((((RATNO13711, RATNO13712, RATNO13713)DUP_2, MOUSE12747)Murinae, SPETR07379, DIPOR01107)Sciurognathi, RABIT15038)Glires)Euarchontoglires, ((VICPA06641, PIGXX18308, (SHEEP18028, BOVIN01193)Bovidae)Cetartiodactyla, (HORSE01285, HORSE01286)DUP_3, (FELCA06632, MUSPF00331)Carnivora, (SORAR00657, SORAR00835)DUP_4)Laurasiatheria)Boreoeutheria, (CHOHO00522, CHOHO02289)DUP_5)Eutheria


In [55]:
alltax = tax.histories[tax.root] #Finds the root of the taxonomy
hoglist =  alltax.geneFamList
for hog in hoglist:
    for geneId in hog.getMemberGenes():
        print(op.mapGeneToXRef(geneId,"protId"))

LOXAF05609
ECHTE05596
RABIT15038
DIPOR01107
MOUSE12747
RATNO13711
RATNO13712
RATNO13713
SPETR07379
MACMU20172
HUMAN00001
HUMAN09883
HUMAN13752
PANTR18725
PONAB04505
CALJA02046
TARSY05187
OTOGA00136
MUSPF00331
FELCA06632
BOVIN01193
SHEEP18028
PIGXX18308
VICPA06641
SORAR00657
SORAR00835
HORSE01285
HORSE01286
CHOHO00522
CHOHO02289


In [37]:
from Bio import SeqIO
omaseqs_dict = SeqIO.index("/cs/research/external/home/naglover/HOGscore/HOG0203542.fasta", "fasta")
line = ""
for hog in hoglist:
    for geneId in hog.getMemberGenes():
        line += ">"+omaseqs_dict[op.mapGeneToXRef(geneId,"protId")].id+"\n"
        line += str(omaseqs_dict[op.mapGeneToXRef(geneId,"protId")].seq)+"\n"
filename = '/cs/research/external/home/naglover/HOGscore/testHOG.fasta'
with open (filename, 'w') as outfile:
    outfile.write(line)

In [38]:
from Bio.Align.Applications import MafftCommandline
from Bio import AlignIO

mafft_cline = MafftCommandline(input=filename) #Create mafft command line
stdout,stderr = mafft_cline() #save mafft output into variable

msa_fasta = '/cs/research/external/home/naglover/HOGscore/testHOGmsa.fasta'
with open(msa_fasta, 'w') as handle:
    handle.write(stdout) #write mafft output in fasta format
    
msa_phy = '/cs/research/external/home/naglover/HOGscore/testHOG.phy'
AlignIO.convert(msa_fasta,"fasta", msa_phy, "phylip-relaxed") #convert mafft output from fasta to phylip

1

In [66]:
from Bio.Phylo.Applications import PhymlCommandline

cmdline = PhymlCommandline("/cs/research/external/home/naglover/programs/PhyML-3.1/PhyML-3.1_linux64", input=msa_phy, datatype='aa', bootstrap=100)
out_log, err_log = cmdline()

In [69]:
import os
cmd = "collapse.py --threshold .50 /cs/research/external/home/naglover/HOGscore/testHOG.phy_phyml_tree.txt"
os.system(cmd)

0

In [101]:
import dendropy
tree1 = dendropy.Tree(stream=open("/cs/research/external/home/naglover/HOGscore/testHOG.phy_phyml_tree-collapse.txt"), schema="newick")
tree2 = dendropy.Tree.get_from_string(newick_short, schema="newick")
diff = tree1.symmetric_difference(tree2)

In [108]:
tree1_edge_length = len(list(tree1.preorder_edge_iter(filter_fn=lambda x: x is not tree1.seed_node.edge)))
tree2_edge_length = len(list(tree2.preorder_edge_iter(filter_fn=lambda x: x is not tree2.seed_node.edge)))

hog_metric = 1.0*diff/(tree1_edge_length + tree2_edge_length)
print(hog_metric)

0.259259259259
